In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, "../src/")
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from handcrafted_features import DocBasedFeatureExtractor, Doc2VecChunkVectorizer, CorpusBasedFeatureExtractor
from utils import get_doc_paths, read_labels
import pickle

raw_docs_dir = "../data/raw_docs/"
labels_dir = "../data/labels/"
extracted_features_dir = "../data/extracted_features/"
feature_extraction_path = '/home/annina/scripts/great_unread_nlp/data/feature_extraction/'

lang = "eng" #"ger", "eng"
doc_paths = get_doc_paths(raw_docs_dir, lang)
sentences_per_chunk = 200
#d2vcv = Doc2VecChunkVectorizer(lang, sentences_per_chunk)
#d2vcv.fit_transform(reduced_doc_paths)

In [2]:
# file_name_mapper = {'eng':['Blackmore_R-D_Lorna-Doone_1869',
#                     'Bulwer-Lytton_Edward_Paul-Clifford_1830',
#                     'Conrad_Joseph_The-Secret-Sharer_1910',
#                     'Parsons_Eliza_The-Castle-of-Wolfenbach_1793',
#                     'Richardson_Samuel_Sir-Charles-Grandison_1753',
#                     'Austen_Jane_Northanger-Abbey_1818',
#                     'Cleland_John_Fanny-Hill_1748',
#                     'Defoe_Daniel_Roxana_1724',
#                     'Fielding_Henry_Amelia_1752',
#                     'Kingsley_Charles_The-Water-Babies_1863',
#                     'Le-Queux_William_The-Invasion-of-1910_1906',
#                     'Surtees_Robert_Jorrocks-Jaunts-and-Jollities_1831'],
#                     'ger': ['Eichendorff_Joseph_Auch-ich-war-in-Arkadien_1832',
#                     'Eichendorff_Joseph_Die-Gluecksritter_1841',
#                     'Eichendorff_Joseph_Libertas-und-ihrer-Freier_1848',
#                     'Eichendorff_Joseph_Viel-Laermen-um-Nichts_1832',
#                     'Goethe_Johann-Wolfgang_Unterhaltungen-deutscher-Ausgewanderten_1795',
#                     'Zschokke_Johann_Addrich-im-Moos_1825',
#                     'Zschokke_Johann_Der-Freihof-von-Aarau_1823',
#                     'Zschokke_Johann_Die-Rose-von-Disentis_1844']}
# for lang in ["eng", "ger"]:
#     file_names = file_name_mapper[lang]
#     print(file_names)
#     doc_paths = ['../data/raw_docs/' + lang + '/' + filename + '.txt' for filename in file_names]
#     print(doc_paths)

    #d2vcv = Doc2VecChunkVectorizer(lang, sentences_per_chunk)
    #d2vcv.fit_transform(reduced_doc_paths)

In [3]:
# all_chunk_based_features = []
# all_book_based_features = []
# all_average_sbert_sentence_embeddings = []
# all_doc2vec_chunk_embeddings = []
# for index, doc_path in enumerate(doc_paths):
#     #if there is an error with chunk_index, rerun the Doc2VecChunkVectorizer for the book where the error occurs
#     fe = DocBasedFeatureExtractor(lang, doc_path, sentences_per_chunk)
#     chunk_based_features, book_based_features, average_sbert_sentence_embeddings, doc2vec_chunk_embeddings = fe.get_all_features()
#     all_chunk_based_features.extend(chunk_based_features)
#     all_book_based_features.append(book_based_features)
#     all_average_sbert_sentence_embeddings.append(average_sbert_sentence_embeddings)
#     all_doc2vec_chunk_embeddings.append(doc2vec_chunk_embeddings)
#     if index > 0 and index % 10 == 0:
#         f = open(feature_extraction_path + lang+ '/features_' + str(index) + '.pkl', 'wb')  
#         pickle.dump([all_chunk_based_features, all_book_based_features, all_average_sbert_sentence_embeddings, all_doc2vec_chunk_embeddings], f, -1)
#         f.close()
#         print('pickle', index)
        
# f = open(feature_extraction_path + lang+ '/all_chunk_based_features' + '.pkl', 'wb')  
# pickle.dump(all_chunk_based_features, f, -1)
# f.close()

# f = open(feature_extraction_path + lang+ '/all_book_based_features' + '.pkl', 'wb')  
# pickle.dump(all_book_based_features, f, -1)
# f.close()

# f = open(feature_extraction_path + lang+ '/all_average_sbert_sentence_embeddings' + '.pkl', 'wb')  
# pickle.dump(all_average_sbert_sentence_embeddings, f, -1)
# f.close()

# f = open(feature_extraction_path + lang+ '/all_doc2vec_chunk_embeddings' + '.pkl', 'wb')  
# pickle.dump(all_doc2vec_chunk_embeddings, f, -1)
# f.close()

In [4]:
f = open(feature_extraction_path + lang + '/all_chunk_based_features' + '.pkl', 'rb')  
all_chunk_based_features = pickle.load(f)
f.close()

f = open(feature_extraction_path + lang + '/all_book_based_features' + '.pkl', 'rb')  
all_book_based_features = pickle.load(f)
f.close()

f = open(feature_extraction_path + lang + '/all_average_sbert_sentence_embeddings' + '.pkl', 'rb')  
all_average_sbert_sentence_embeddings = pickle.load(f)
f.close()

f = open(feature_extraction_path + lang + '/all_doc2vec_chunk_embeddings' + '.pkl', 'rb')  
all_doc2vec_chunk_embeddings = pickle.load(f)
f.close()

# f = open(feature_extraction_path + lang + '/all_corpus_based_features_' + '.pkl', 'rb')  
# all_corpus_based_features = pickle.load(f)
# f.close()

In [5]:
cbfe = CorpusBasedFeatureExtractor(lang, doc_paths, all_average_sbert_sentence_embeddings, all_doc2vec_chunk_embeddings)
all_corpus_based_features = cbfe.get_all_features(k=100)

print(all_corpus_based_features.shape)#ger 30: (547, 726), 50: (547, 1181) #eng 30: (606, 907)

all_corpus_based_features.columns

f = open(feature_extraction_path + lang+ '/all_corpus_based_features' + '.pkl', 'wb')  
pickle.dump(all_corpus_based_features, f, -1)
f.close()

100%|██████████| 606/606 [00:47<00:00, 12.69it/s]


(606, 907)


In [6]:
import os
import pandas as pd

book_df = pd.DataFrame(all_book_based_features)
book_df = book_df.merge(all_corpus_based_features, on="book_name")
book_and_averaged_chunk_df = book_df.merge(pd.DataFrame(all_chunk_based_features).groupby("book_name").mean().reset_index(drop=False), on="book_name")

chunk_df = pd.DataFrame(all_chunk_based_features)
chunk_and_copied_book_df = chunk_df.merge(pd.DataFrame(all_book_based_features), on="book_name")
chunk_and_copied_book_df = chunk_and_copied_book_df.merge(all_corpus_based_features, on="book_name")

#os.makedirs(f"{extracted_features_dir}/{lang}", exist_ok=True)
book_df.to_csv(f"{extracted_features_dir}/{lang}/book_df.csv", index=False)
book_and_averaged_chunk_df.to_csv(f"{extracted_features_dir}/{lang}/book_and_averaged_chunk_df.csv", index=False)
chunk_df.to_csv(f"{extracted_features_dir}/{lang}/chunk_df.csv", index=False)
chunk_and_copied_book_df.to_csv(f"{extracted_features_dir}/{lang}/chunk_and_copied_book_df.csv", index=False)

chunk_and_copied_book_df.shape

(15330, 1800)